In [121]:
# upload data
# libraries extraction
import glob 
import os 
import pandas as pd
import numpy as np 
from scipy.stats import ttest_ind 
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt 

extension = 'csv'
#import pandas as pd

In [122]:

df = pd.read_csv('C:/Users/Betty Reinhardova/Desktop/University/Thesis - Ms/data/data_thesis.csv')
df.head(3)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,...,revt,sale,seq,wcap,xrd,costat,mkvalt,gsector,sic,state
0,1004,31/05/2013,2012,INDL,C,D,STD,AIR,AAR CORP,USD,...,2167.1,2167.1,918.6,644.7,NaN,A,790.0029,20.0,5080,IL
1,1004,31/05/2014,2013,INDL,C,D,STD,AIR,AAR CORP,USD,...,2035.0,2035.0,999.5,714.8,NaN,A,961.3080,20.0,5080,IL
2,1004,31/05/2015,2014,INDL,C,D,STD,AIR,AAR CORP,USD,...,1594.3,1594.3,845.1,542.1,NaN,A,1046.3954,20.0,5080,IL


In [123]:
#creating unique ID
df['id'] = df.index + 1
df.head()
df['id'].count()

56013

In [124]:
#remove rows that do not have tic code
#filter out tic codes that are shorter than 4 or five letters = no special characters...
#remove sic in range of 4900-4999 (including)
#remove sic in range of 6000-6999 (including)

df.dropna(subset=['tic'], inplace=True)
df.dropna(subset=['sic'], inplace=True)
df['tic'] = df['tic'].astype("string")
print(df['tic'].count())
print(df['tic'].str.len())
#FILTER TIC ONLY 4-5 LETTERS
#print(df.dtypes) 
#print(((df['tic'].str.len() ==5) | (df['tic'].str.len() ==4)).count())
#df.drop(df[(df['tic'].str.len() ==5) | (df['tic'].str.len() ==4)].index, inplace=True)

#print(df['tic'].str.match('[A-z]{4,5}'))
df = df[(df['tic'].str.match('[A-z]{4,5}'))]

df.drop(df[(df['sic'] >= 4900) & (df['sic'] <= 4999)].index, inplace=True)
df.drop(df[(df['sic'] >= 6000) & (df['sic'] <= 6999)].index, inplace=True)
#df['id'].count()
df.drop(df.columns[[3,4,5,6,10,32]], axis=1, inplace=True)

df.shape[0]

55998
0        3
1        3
2        3
3        3
4        3
        ..
56008    4
56009    4
56010    4
56011    5
56012    5
Name: tic, Length: 55998, dtype: Int64


19802

In [125]:
df['xrd'].fillna(0,inplace=True)
df['at'].fillna(0,inplace=True)
df['wcap'].fillna(0,inplace=True)
df['ebit'].fillna(0,inplace=True)
df['re'].fillna(0,inplace=True)
df['sale'].fillna(0,inplace=True)
df['derac'].fillna(0,inplace=True)
df['deralt'].fillna(0,inplace=True)
df['derlc'].fillna(0,inplace=True)
df['derllt'].fillna(0,inplace=True)
df['cidergl'].fillna(0,inplace=True)
df['lt'].fillna(0,inplace=True)

In [126]:
df['rd intensity'] = df['xrd'] / df['at']
#create new variable - R&D intensity = R&D expenditure / TA
df['financial distress prob'] = (1.2 * df['wcap'] + 1.4 * df['re'] + 3.3 * df['ebit'] + 0.999 * df['sale'])/df['at']
#create new variable - Altman Z-score = (1.2 x WC + 1.4 x RE + 3.3 x EBIT + 0.999 x Sales)/TA -> score lower than 1.65 = risky business -> please check the range and go back to thesis page 26
df['hedging intensity'] = (df['derac'] + df['deralt'] + df['derlc']) + df['derllt'] / df['at']

df['hedging intensity'].fillna(0,inplace=True)
df['rd intensity'].fillna(0,inplace=True)
df['financial distress prob'].fillna(0,inplace=True)
#WHY IS IT NOT WORKING? -> IN DESCRIPTIVES IT LEAVES ME WITH NAN

#df[['rd intensity', 'hedging intensity']].replace(np.nan, 0)


df.head(10)
#df.dtypes


,gvkey,datadate,fyear,tic,conm,curcd,at,cidergl,derac,deralt,...,wcap,xrd,mkvalt,gsector,sic,state,id,rd intensity,financial distress prob,hedging intensity
14,1050,31/12/2013,2013,CECO,CECO ENVIRONMENTAL CORP,USD,348.536,0.000,0.0,0.0,...,65.439,0.0,413.2812,20.0,3564,TX,15,0.0,1.014667,0.000
15,1050,31/12/2014,2014,CECO,CECO ENVIRONMENTAL CORP,USD,414.365,0.000,0.0,0.0,...,67.616,0.0,408.1892,20.0,3564,TX,16,0.0,1.066187,0.000
16,1050,31/12/2015,2015,CECO,CECO ENVIRONMENTAL CORP,USD,598.819,0.000,0.0,0.0,...,80.283,0.0,260.4902,20.0,3564,TX,17,0.0,0.922141,0.000
17,1050,31/12/2016,2016,CECO,CECO ENVIRONMENTAL CORP,USD,498.634,0.312,0.0,0.0,...,66.620,0.0,476.5599,20.0,3564,TX,18,0.0,1.088944,0.000
18,1050,31/12/2017,2017,CECO,CECO ENVIRONMENTAL CORP,USD,438.549,0.180,0.0,0.3,...,66.091,0.0,177.3441,20.0,3564,TX,19,0.0,0.860361,0.300
19,1050,31/12/2018,2018,CECO,CECO ENVIRONMENTAL CORP,USD,392.582,0.076,0.0,0.5,...,56.979,0.0,235.0080,20.0,3564,TX,20,0.0,0.891030,0.500
20,1050,31/12/2019,2019,CECO,CECO ENVIRONMENTAL CORP,USD,408.637,-0.574,0.0,0.5,...,64.310,0.0,269.1494,20.0,3564,TX,21,0.0,0.974613,0.500
56,1084,31/12/2013,2013,WDDD,WORLDS INC,USD,0.328,0.000,0.0,0.0,...,-4.009,0.0,13.9815,50.0,7370,MA,57,0.0,-200.418293,0.429
57,1084,31/12/2014,2014,WDDD,WORLDS INC,USD,0.028,0.000,0.0,0.0,...,-4.604,0.0,14.0435,50.0,7370,MA,58,0.0,-2098.985714,0.427
58,1084,31/12/2015,2015,WDDD,WORLDS INC,USD,0.026,0.000,0.0,0.0,...,-5.142,0.0,3.3654,50.0,7370,MA,59,0.0,-2498.034615,0.416


In [127]:
#df['dummy hedge'] = pd.get_dummies(df['hedging intensity'])
dummy_hedge = df['hedge group'] = (df['hedging intensity'] > 0).astype(int)
hedger = dummy_hedge==1
nonhedger = dummy_hedge==0
#hedger = [df['hedge group'] == 1]
#nonhedger = [df['hedge group] == 0]
df.head(10)



,gvkey,datadate,fyear,tic,conm,curcd,at,cidergl,derac,deralt,...,xrd,mkvalt,gsector,sic,state,id,rd intensity,financial distress prob,hedging intensity,hedge group
14,1050,31/12/2013,2013,CECO,CECO ENVIRONMENTAL CORP,USD,348.536,0.000,0.0,0.0,...,0.0,413.2812,20.0,3564,TX,15,0.0,1.014667,0.000,0
15,1050,31/12/2014,2014,CECO,CECO ENVIRONMENTAL CORP,USD,414.365,0.000,0.0,0.0,...,0.0,408.1892,20.0,3564,TX,16,0.0,1.066187,0.000,0
16,1050,31/12/2015,2015,CECO,CECO ENVIRONMENTAL CORP,USD,598.819,0.000,0.0,0.0,...,0.0,260.4902,20.0,3564,TX,17,0.0,0.922141,0.000,0
17,1050,31/12/2016,2016,CECO,CECO ENVIRONMENTAL CORP,USD,498.634,0.312,0.0,0.0,...,0.0,476.5599,20.0,3564,TX,18,0.0,1.088944,0.000,0
18,1050,31/12/2017,2017,CECO,CECO ENVIRONMENTAL CORP,USD,438.549,0.180,0.0,0.3,...,0.0,177.3441,20.0,3564,TX,19,0.0,0.860361,0.300,1
19,1050,31/12/2018,2018,CECO,CECO ENVIRONMENTAL CORP,USD,392.582,0.076,0.0,0.5,...,0.0,235.0080,20.0,3564,TX,20,0.0,0.891030,0.500,1
20,1050,31/12/2019,2019,CECO,CECO ENVIRONMENTAL CORP,USD,408.637,-0.574,0.0,0.5,...,0.0,269.1494,20.0,3564,TX,21,0.0,0.974613,0.500,1
56,1084,31/12/2013,2013,WDDD,WORLDS INC,USD,0.328,0.000,0.0,0.0,...,0.0,13.9815,50.0,7370,MA,57,0.0,-200.418293,0.429,1
57,1084,31/12/2014,2014,WDDD,WORLDS INC,USD,0.028,0.000,0.0,0.0,...,0.0,14.0435,50.0,7370,MA,58,0.0,-2098.985714,0.427,1
58,1084,31/12/2015,2015,WDDD,WORLDS INC,USD,0.026,0.000,0.0,0.0,...,0.0,3.3654,50.0,7370,MA,59,0.0,-2498.034615,0.416,1


In [128]:
# CONTROL VARIABLES
df['size'] = np.log(df['at'])
# SIZE = natural log of bv assets
df['fp'] = df['ebitda'] / df['at'] 
# FP = EBITDA / TA
df['ast'] = df['ppent'] / df['at']
#AST = PP&E / TA 
df['mb'] = df['mkvalt']/ (df['at'] - df['lt']) 
# M/B = market value of equity+ book value of debt / TA   CHECK
df['leve'] = df['lt']
# Leve
df['sector'] = df['sic']
# SECTOR = fixed effect
df['time'] = df['fyear']
# TIME
df['defr'] = df['financial distress prob']
# Def R = altman zcosre
df['size'].fillna(0,inplace=True)
df['fp'].fillna(0,inplace=True)
df['ast'].fillna(0,inplace=True)
df['mb'].fillna(0,inplace=True)
df['leve'].fillna(0,inplace=True)
df['defr'].fillna(0,inplace=True)
df.head(10)

c:\Users\Betty Reinhardova\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,gvkey,datadate,fyear,tic,conm,curcd,at,cidergl,derac,deralt,...,hedging intensity,hedge group,size,fp,ast,mb,leve,sector,time,defr
14,1050,31/12/2013,2013,CECO,CECO ENVIRONMENTAL CORP,USD,348.536,0.000,0.0,0.0,...,0.000,0,5.853742,0.073573,0.062160,2.425274,178.130,3564,2013,1.014667
15,1050,31/12/2014,2014,CECO,CECO ENVIRONMENTAL CORP,USD,414.365,0.000,0.0,0.0,...,0.000,0,6.026747,0.085915,0.045759,2.252401,233.141,3564,2014,1.066187
16,1050,31/12/2015,2015,CECO,CECO ENVIRONMENTAL CORP,USD,598.819,0.000,0.0,0.0,...,0.000,0,6.394959,0.075433,0.075116,1.063134,353.798,3564,2015,0.922141
17,1050,31/12/2016,2016,CECO,CECO ENVIRONMENTAL CORP,USD,498.634,0.312,0.0,0.0,...,0.000,0,6.211872,0.112746,0.054689,2.507128,308.552,3564,2016,1.088944
18,1050,31/12/2017,2017,CECO,CECO ENVIRONMENTAL CORP,USD,438.549,0.180,0.0,0.3,...,0.300,1,6.083472,0.063995,0.053358,0.950555,251.980,3564,2017,0.860361
19,1050,31/12/2018,2018,CECO,CECO ENVIRONMENTAL CORP,USD,392.582,0.076,0.0,0.5,...,0.500,1,5.972745,0.069626,0.056549,1.316129,214.022,3564,2018,0.891030
20,1050,31/12/2019,2019,CECO,CECO ENVIRONMENTAL CORP,USD,408.637,-0.574,0.0,0.5,...,0.500,1,6.012827,0.073980,0.070676,1.394434,215.620,3564,2019,0.974613
56,1084,31/12/2013,2013,WDDD,WORLDS INC,USD,0.328,0.000,0.0,0.0,...,0.429,1,-1.114742,-12.103659,0.000000,-3.493628,4.330,7370,2013,-200.418293
57,1084,31/12/2014,2014,WDDD,WORLDS INC,USD,0.028,0.000,0.0,0.0,...,0.427,1,-3.575551,-29.142857,0.000000,-3.050282,4.632,7370,2014,-2098.985714
58,1084,31/12/2015,2015,WDDD,WORLDS INC,USD,0.026,0.000,0.0,0.0,...,0.416,1,-3.649659,-29.576923,0.000000,-0.654492,5.168,7370,2015,-2498.034615


In [129]:
# descriptive statistics 
from IPython.display import Image
descriptives_stat = np.round(df.describe(include='number'),2).transpose()[['count','mean','std','min','max']]
descriptives_stat.to_csv('descriptives_statistics.csv')
#WHY DOESN'T IT PRINT WHAT I TOLD IT TO DO?
descriptives_stat.to_latex('summary stats.tex')
#Image(width=600, filename='Descriptives Statistics Table (LaTex).png')
descriptives_stat 

,count,mean,std,min,max
gvkey,19802.0,81869.84,76526.87,1050.00,351590.00
fyear,19802.0,2016.08,2.06,2012.00,2019.00
at,19802.0,4095.39,22520.24,0.00,547704.82
cidergl,19802.0,-0.54,85.18,-4311.59,4785.96
derac,19802.0,10.84,194.45,-28.47,14037.00
deralt,19802.0,10.39,124.34,-7.00,4918.20
derhedgl,5796.0,-3.98,127.71,-4480.42,2561.78
derlc,19802.0,13.62,223.28,-0.26,11989.67
derllt,19802.0,10.67,122.80,-8.66,5197.12
dlc,18796.0,246.52,2379.35,0.00,102762.57


In [130]:
sic_codes = df['sic']
rd_investment = df['rd intensity']

bar_chart = rd_investment.plot.bar(sic_codes, rot=0, color= '#95b8d1')
bar_chart.set_title('R&D Investmetn distribution across Sectors')
bar_chart.set_ylable 

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000002C716BF3E50> (for post_execute):


KeyboardInterrupt: 

In [ ]:
#T-test1 -> hedging and financial distress
# I need to lag all controls one year back because of the decision 
#gonna run paired sample t-test -> because firms could have start using hedging during the years? -> cannot be because arrays are not equal = we asusme that the firms that hedge do differ form firms that do not 

ttest1 = ttest_ind(hedger['financial distress prob'],nonhedger['financial distress prob'])
#ttest1_paired = ttest_rel(hedger['financial distress prob'],nonhedger['financial distress prob'])


#T-test2 -> hedging and R&D investment

ttest2 = ttest_ind(hedger['rd intensity'],nonhedger['rd intensity'])
#ttest2_paired = ttest_rel(hedger['rd intensity'],nonhedger['rd intensity'])
#WHY DO I GET NAN VALUES? 

KeyError: 'financial distress prob'

In [ ]:
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
import statsmodels as stm 
import statsmodels.graphics.tsaplots as tsap
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_white
import seaborn as sns

In [ ]:
A = 6
plt.rc('figure', figsize=[46.82 * .5 ** (.5 * A), 33.11 * .5**(.5 *A)])
plt.rc('figure', dpi=141)
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
#setting plot parameters

ax = plt.axes()
altman_z = df['financial distress prob']
rd_intensity = df['rd intensity']
bp = ax.boxplot(df, widths=0.6, patch_artist=True)
ax.set_title('Hedgers vs Nonhedgers financial distress probability', fontsize=16)
ax.set_ylabel('Altman Z-score')
xticklabels = df['hedge group']
xticklabels = [lable.title() for label in xticklabels]
#I understand what I am doing but dont understand why it does not work


ax.set_xticklabels(xticklabels)
# hide x-axis major ticks => WHY
ax.tick_params(axis='x', which = 'major', length=0)
xticks = [0.5] + [x + 0.5 for x in ax.get_xticks()]
ax.set_xticks(xticks, minor=True)
ax.tick_params(axis='x', which='minor', lenght=3, width=1)

for i, significnat_combination in enumerate(significnat_combination):
    x1 = significnat_combination[0][0] 
    x2= significnat_combination[0][1]
    level = len(significnat_combination) -i
    bar_height = (y_range *0.07 * level) + top 
    bar_tips = bar_height - (y_range * 0.02)
    plt.plot([x1,x1,x2,x2],[bar_tips,bar_height,bar_height,bar_tips], Llw=1, c='K')
    p = significnat_combination[1]
    if p < 0.001:
        sig_symbol = '***'
    elif p < 0.01:
        sig_symbol = '**'
    elif p < 0.05:
        sig_symbol = '*'
    text_height = bar_height + (y_range * 0.01)
    plt.text((x1+x2)*0.5, text_height, sig_symbol, ha='center', va='bottom', c='k')

plt.show()
#rowannicholls.github.io/python/graphs/ax_based/boxplots_significance.html

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 825.202x583.564 with 1 Axes>

In [ ]:
#Hausman test = to decided whether pooled or fix model 

In [ ]:
#OLS = pooled 1
sns.scatterplot(x=df['hedging intensity'], y=df['financial distress prob'], hue=df['sic'])
plt.show()
#not workign cause of string -> is it becasue I do have Nan values? 

x_var_ols1 =
y_var_ols2 =
pooled_ols_1 = sm.OLS(endog=y_var_ols2,exog=x_var_ols1)
pooled_ols_1_results = pooled_ols_1.fit()
print(pooled_ols_1_results.summary()) 

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 825.202x583.564 with 1 Axes>

In [ ]:
#OLS = pooled 2 

In [ ]:
#Robustness test 